In [ ]:
# TODO UPDATE THIS OLD MESSY SCRIPT

In [ ]:


# FOR CLEANING SCRIPT:

# ensure each asset does not appear before first date nor after last date
# cmc_ids = np.unique(panel_df.cmc_id.values)
# for cmc_id in cmc_ids:
#     print(cmc_id)
#     first_date = cw_df[cw_df.cmc_id==cmc_id].first_date_cmc.values[0]
#     last_date  = cw_df[cw_df.cmc_id==cmc_id].last_date_cmc.values[0]
#     assert(0==panel_df[(panel_df.cmc_id==cmc_id)&(panel_df.date<first_date)].shape[0])
#     assert(0==panel_df[(panel_df.cmc_id==cmc_id)&(panel_df.date>last_date)].shape[0])

# ensure each asset has consecutive data, interpolate where needed with forward fill

# group the data by cmc_id to loop over
grouped = panel_df.groupby('cmc_id')

# interate through each cmc_id
dfs = []
for name, group in grouped:
    # find the first and last dates for the current id
    first_date = group['date'].min()
    last_date  = group['date'].max()

    # create a new dataframe with all the possible combinations of cmc_id and date
    dates = pd.date_range(first_date, last_date)
    index = pd.MultiIndex.from_product([[name], dates], names=['cmc_id', 'date'])
    full_df = pd.DataFrame(index=index).reset_index()

    # merge the full dataframe with the original dataframe to fill in missing values with NaNs
    merged_df = pd.merge(full_df, group, on=['cmc_id', 'date'], how='left')

    # interpolate the missing values using forward fill for up to 7 consecutive observations
    interpolated_df = merged_df.fillna(method='ffill', limit=21)

    # Check if there are any missing values in the remaining columns for the current id and date range
    if interpolated_df.isnull().values.any():
        print(f"ID {name} has missing values in the given date range, precisely: {int(interpolated_df.isnull().sum().sum()/3)}.")
        break

    # combine    
    dfs.append(interpolated_df)

# Combine all the dataframes and drop the 'cmc_id' index level
result_df = pd.concat(dfs).reset_index()




In [41]:
import pandas as pd
import numpy as np

In [42]:
# IMPORT DATA
cw_df = pd.read_csv('../3-data/raw/cmc_token_universe.csv')
cmc_df   = pd.read_csv('../3-data/raw/cmc_token_covars_panel.csv')
panel_df = pd.read_csv('../3-data/raw/cmc_price_vol_mcap_panel.csv')
macro_df = pd.read_csv('../3-data/raw/cmc_macro_timeseries.csv')
ex_df    = pd.read_csv('../3-data/raw/cmc_exchange_panel.csv')
ex_covar_df = pd.read_csv('../3-data/raw/cmc_exchange_covar.csv')

In [43]:
# CLEAN TOKEN COVARS

# Merge token covars from crosswalk data onto the cmc df
cmc_df = cmc_df.merge(cw_df[['cmc_id', 'cmc_first_date', 'cmc_last_date', 'platform_cmc_slug']],
                      on='cmc_id',
                      how='outer',
                      validate='many_to_one')

# Rename columns to standard convention (with data source name in it)
cmc_df   = cmc_df.rename(columns = {'num_market_pairs': 'num_market_pairs_cmc', 
                                    'max_supply': 'max_supply_cmc',
                                    'circulating_supply': 'circulating_supply_cmc',
                                    'total_supply': 'total_supply_cmc',
                                    'cmc_rank': 'rank_cmc',
                                    'cmc_first_date': 'first_date_cmc',
                                    'cmc_last_date': 'last_date_cmc'})

# Obtain unique list of all tags in tags column
all_tags = []
for tags in cmc_df.tags.values:
    tags = tags.strip('\'][\'').split(', ')
    tags = [tag.replace('\'', '') for tag in tags]
    all_tags.append(tags)
unique_tags = list(set([tag for tags in all_tags for tag in tags]))

# Determine the number of appearances of each tag
tag_counts_dict = {}
for tag in unique_tags:
    tag_counts_dict[tag] = 0
for tag in cmc_df.tags.values:
    tag_list = tag.strip('\'][\'').split(', ')
    if len(tag_list) >= 1:
        tag_list = [tag.replace('\'', '') for tag in tag_list]
        for tag in tag_list:
            tag_counts_dict[tag] += 1
    
# Create a tag to column name dictionary
tag_col_dict = {'platform': 'platform_cmc',
                'mineable': 'mineable_cmc',
                'pow': 'pow_cmc',
                'powt': 'pow_cmc',
                'hybrid-dpow-pow': 'pow_cmc',
                'hybrid-pos-lpos': 'pos_cmc',
                'pos': 'pos_cmc',
                'hybrid-pos-pop': 'pos_cmc',
                'dpos': 'pos_cmc',
                'rpos': 'pos_cmc',
                'placeholder-ventures-portfolio': 'portfolio_placeholder_cmc',
                'dragonfly-capital-portfolio': 'portfolio_dragonfly_cmc',
                'three-arrows-capital-portfolio': 'portfolio_3arrows_cmc',
                'electric-capital-portfolio': 'portfolio_electric_cap_cmc',
                'coinbase-ventures-portfolio': 'portfolio_coinbase_cmc',
                'winklevoss-capital-portfolio': 'portfolio_winklevoss_cmc',
                'winklevoss-capital': 'portfolio_winklevoss_cmc',
                'a16z-portfolio': 'portfolio_a16z_cmc',
                'multicoin-capital-portfolio': 'portfolio_multicoin_cap_cmc',
                'polychain-capital-portfolio': 'portfolio_polychain_cmc',
                'pantera-capital-portfolio': 'portfolio_pantera_cmc',
                'dcg-portfolio': 'portfolio_dcg_cmc',
                'alameda-research-portfolio': 'portfolio_alameda_cmc',
                'paradigm-portfolio': 'portfolio_paradigm_cmc',
                'galaxy-digital-portfolio': 'portfolio_galaxy_digital_cmc',
                'usv-portfolio': 'portfolio_usv_cmc',
                'ethereum': 'ecosystem_eth_cmc',
                'ethereum-ecosystem': 'ecosystem_eth_cmc',
                'algorand-ecosystem': 'ecosystem_algorand_cmc',
                'polkadot': 'ecosystem_polkadot_cmc',
                'cosmos': 'ecosystem_cosmos_cmc',
                'cosmos-ecosystem': 'ecosystem_cosmos_cmc',
                'terra-ecosystem': 'ecosystem_terra_cmc',
                'solana-ecosystem': 'ecosystem_solana_cmc',
                'binance-smart-chain-ecosystem': 'ecosystem_binance_cmc',
                'polygon-ecosystem': 'ecosystem_polygon_cmc',
                'cardano-ecosystem': 'ecosystem_cardano_cmc',
                'avalanche-ecosystem': 'ecosystem_avalanche_cmc',
                'near-protocol-ecosystem': 'ecosystem_near_cmc',
                'stablecoin-asset-backed': 'sector_stablecoin_cmc',
                'asset-backed-stablecoin': 'sector_stablecoin_cmc',
                'stablecoin-algorithmically-stabilized': 'sector_stablecoin_cmc',
                'algorithmic-stablecoin': 'sector_stablecoin_cmc',
                'stablecoin': 'sector_stablecoin_cmc',
                'derivatives': 'sector_defi_cmc',
                'defi-index': 'sector_defi_cmc',
                'options': 'sector_defi_cmc',
                'yield-aggregator': 'sector_defi_cmc',
                'lending-borowing': 'sector_defi_cmc',
                'yield-farming': 'sector_defi_cmc',
                'asset-management': 'sector_defi_cmc',
                'defi': 'sector_defi_cmc',
                'dao': 'sector_dao_cmc',
                'atomic-swaps': 'sector_payments_cmc',
                'payments': 'sector_payments_cmc',
                'layer-2': 'sector_infastructure_cmc',
                'oracles': 'sector_infrastructure_cmc',
                'interoperability': 'sector_infrastructure_cmc',
                'rollups': 'sector_infastructure_cmc',
                'zero-knowledge-proofs': 'sector_infastructure_cmc',
                'scaling': 'sector_infastructure_cmc',
                'wallet': 'sector_infastructure_cmc',
                'smart-contracts': 'sector_infastructure_cmc',
                'staking': 'sector_infastructure_cmc',
                'identity': 'sector_infrastructure_cmc',
                'fan-token': 'sector_social_cmc',
                'social-money': 'sector_social_cmc',
                'social-token': 'sector_social_cmc',
                'communications-social-media': 'sector_social_cmc',
                'distributed-computing': 'sector_computing_cmc',
                'storage': 'sector_computing_cmc',
                'enterprise-solutions': 'sector_enterprise_cmc',
                'privacy': 'sector_privacy_cmc',
                'agriculture': 'sector_agriculture_cmc',
                'sports': 'sector_games_cmc',
                'gaming': 'sector_games_cmc',
                'play-to-earn': 'sector_games_cmc',
                'gambling': 'sector_games_cmc',
                'energy': 'sector_energy_cmc',
                'cybersecurity': 'sector_cybersecurity_cmc',
                'store-of-value': 'sector_sov_cmc',
                'ai-big-data': 'sector_ai_cmc',
                'crowdsourcing': 'sector_crowdsourcing_cmc', 
                'crowdfunding': 'sector_crowdsourcing_cmc',
                'prediction-markets': 'sector_crowdsourcing_cmc',
                'memes': 'sector_memes_cmc',
                'entertainment': 'sector_media_cmc',
                'media': 'sector_media_cmc',
                'music': 'sector_media_cmc',
                'marketplace': 'sector_market_cmc',
                'e-commerce': 'sector_market_cmc',
                'centralized-exchange': 'sector_cex_cmc',
                'medium-of-exchange': 'sector_moe_cmc',
                'dex': 'sector_dex_cmc',
                'decentralized-exchange ': 'sector_dex_cmc',
                'amm': 'sector_dex_cmc',
                'collectibles-nfts': 'sector_nft_cmc',
                'real-estate': 'sector_real_estate_cmc',
                'metaverse': 'sector_metaverse_cmc',
                'education': 'sector_education_cmc'}

# Add the values in the dictionary as new columns with zeros
cmc_df = cmc_df.reindex(columns=[*cmc_df.columns.tolist(), 
                                 *list(set(tag_col_dict.values()))], fill_value=0)

# Update the new columns with 1 if contained in tags column
n = cmc_df.shape[0]
keys = list(tag_col_dict.keys())
for i in range(n):
    # Obtain tags
    tags = all_tags[i]
    
    # Update columns
    if tags != ['']:
        for tag in tags:
            if tag in keys:
                col = tag_col_dict[tag]
                cmc_df.at[i, col] = 1
                
# Drop tags column
cmc_df = cmc_df.drop('tags', axis=1)

# Convert columns to correct data type
cmc_df['date'] = pd.to_datetime(cmc_df.date, format='%Y-%m-%d', utc=False)
cmc_df['first_date_cmc'] = pd.to_datetime(cmc_df.first_date_cmc, format='%Y-%m-%d', utc=False).dt.date
cmc_df['last_date_cmc'] = pd.to_datetime(cmc_df.last_date_cmc, format='%Y-%m-%d', utc=False).dt.date

# Use platform slug column to update platform values
cmc_df.loc[cmc_df.platform_cmc_slug == 'polygon', 'ecosystem_polygon_cmc'] = 1
cmc_df.loc[cmc_df.platform_cmc_slug == 'cardano', 'ecosystem_cardano_cmc'] = 1
cmc_df.loc[cmc_df.platform_cmc_slug == 'bnb', 'ecosystem_binance_cmc'] = 1
cmc_df.loc[cmc_df.platform_cmc_slug == 'solana', 'ecosystem_solana_cmc'] = 1
cmc_df.loc[cmc_df.platform_cmc_slug == 'avalanche', 'ecosystem_avalanche_cmc'] = 1
cmc_df.loc[cmc_df.platform_cmc_slug == 'terra-luna', 'ecosystem_terra_cmc'] = 1
cmc_df.loc[cmc_df.platform_cmc_slug == 'algorand', 'ecosystem_algorand_cmc'] = 1
cmc_df.loc[cmc_df.platform_cmc_slug == 'arbitrum-ethereum', 'ecosystem_eth_cmc'] =1
cmc_df.loc[cmc_df.platform_cmc_slug == 'cosmos', 'ecosystem_cosmos_cmc'] = 1

# Drop platform slug column
cmc_df = cmc_df.drop('platform_cmc_slug', axis=1)

# Form list of data columns to work with
data_cols = list(cmc_df.columns.values)
data_cols.remove('date')
data_cols.remove('cmc_id')

# Set column order
cmc_df = cmc_df[['date', 'cmc_id'] + data_cols]

# Drop rows
cmc_df = cmc_df[(cmc_df.date.dt.year >= 2015) & (cmc_df.date <= '2022-01-02')]

# Trim values
cmc_df.loc[cmc_df.max_supply_cmc < 0, 'max_supply_cmc'] = np.nan
cmc_df.loc[cmc_df.max_supply_cmc < 0, 'max_supply_cmc'] = np.nan

# Drop duplicated rows across id columns
cmc_df = cmc_df.drop_duplicates(subset=['date', 'cmc_id'])

# Sort values and reset index
cmc_df = cmc_df.sort_values(by=['date', 'cmc_id'], 
                            ignore_index=True)

In [138]:
# CLEAN PANEL

# Rename columns to standard convention (with data source name in it)
panel_df = panel_df.rename(columns = {'usd_per_token': 'usd_per_token_cmc',
                                      'usd_mcap': 'usd_mcap_cmc',
                                      'usd_volume_24h': 'usd_volume_24h_cmc'})

# Convert columns to correct data type
panel_df['date'] = pd.to_datetime(panel_df.date, format='%Y-%m-%d', utc=False)

# Set column order
panel_df = panel_df[['date', 'cmc_id', 'usd_per_token_cmc', 'usd_mcap_cmc', 'usd_volume_24h_cmc']]

# Drop rows
panel_df = panel_df[(panel_df.date.dt.year >= 2015) & (panel_df.date <= '2022-01-02')]
panel_df = panel_df[panel_df.index.isin(panel_df[['usd_per_token_cmc', 
                                                  'usd_mcap_cmc', 
                                                  'usd_volume_24h_cmc']].dropna(how='all').index.values)]

# Confirm no missings in the df
assert(panel_df.isnull().sum().values.sum() == 0)

# Form list of data columns to work with
data_cols = list(panel_df.columns.values)
data_cols.remove('date')
data_cols.remove('cmc_id')

# Set negative values to missing and too large values to missing
for col in data_cols:
    panel_df.loc[panel_df[col] < 0, col] = np.nan
    panel_df.loc[panel_df[col] > 2e12, col] = np.nan

# Confirm no negative values nor too large values
for col in data_cols:
    assert(0 <= np.nanmin(panel_df[col].values))
    assert(2e12 >= np.nanmax(panel_df[col].values))

# Drop duplicated rows across id columns
panel_df = panel_df.drop_duplicates(subset=['date', 'cmc_id'])

# Sort values and reset index
panel_df = panel_df.sort_values(by=['date', 'cmc_id'], 
                                ignore_index=True)


In [139]:
# MERGE
panel_df = panel_df.merge(cmc_df,
                          on=['date', 'cmc_id'],
                          how='left',
                          validate='one_to_one')

In [140]:
# CLEAN CW

# Convert column names
cw_df = cw_df.rename(columns = {'cmc_symbol': 'symbol_cmc',
                                'name': 'name_cmc',
                                'cmc_slug': 'slug_cmc'})

# Drop columns from crosswalk
cw_df = cw_df[['cmc_id', 'symbol_cmc', 'name_cmc', 'slug_cmc']]

# Confirm ID column is unique
assert(cw_df.shape[0] == len(np.unique(cw_df.cmc_id.values)))

# Sort values and reset index
cw_df = cw_df.sort_values(by='cmc_id', ignore_index=True)



In [141]:
# CLEAN MACRO VARS

# Clean date column
ex_df['date'] = pd.to_datetime(ex_df.date, format='%Y-%m-%d', utc=False).dt.date
macro_df['date'] = pd.to_datetime(macro_df.date, format='%Y-%m-%d', utc=False).dt.date

# Reshape exchange volume and market pairs from long to wide
us_ex = ['binance-us', 'coinbase-exchange', 'ftx-us',  'gemini', 'kraken', 'kucoin']
ex_df = ex_df[~((ex_df.exchange_volume_24h == -1) & (ex_df.num_market_pairs == -1))].reset_index(drop=True)

ex_df = ex_df.merge(ex_covar_df[['exchange_id', 'exchange_slug']],
                    how='inner',
                    on='exchange_id',
                    validate='many_to_one')
ex_df = ex_df.drop('exchange_id', axis=1)
ex_df = ex_df[ex_df.exchange_slug.isin(us_ex)]
ex_df = ex_df.rename(columns={'exchange_volume_24h': 'volume_24h'})
ex_df = ex_df[~ex_df.exchange_slug.isin(['b2bx', 'dodo', 'demex'])]
ex_vol_df = ex_df.pivot(index='date', columns='exchange_slug', values='volume_24h').reset_index()
ex_pairs_df = ex_df.pivot(index='date', columns='exchange_slug', values='num_market_pairs').reset_index()
ex_vol_df = ex_vol_df.rename(columns={'binance-us': 'exchange_volume_binance_us_cmc',
                                      'coinbase-exchange': 'exchange_volume_coinbase_cmc',
                                      'ftx-us': 'exchange_volume_ftx_us_cmc',
                                      'gemini': 'exchange_volume_gemini_cmc',
                                      'kraken': 'exchange_volume_kraken_cmc',
                                      'kucoin': 'exchange_volume_kucoin_cmc'})
ex_pairs_df = ex_pairs_df.rename(columns={'binance-us':        'exchange_pairs_binance_us_cmc',
                                          'coinbase-exchange': 'exchange_pairs_coinbase_cmc',
                                          'ftx-us': 'exchange_pairs_ftx_us_cmc',
                                          'gemini': 'exchange_pairs_gemini_cmc',
                                          'kraken': 'exchange_pairs_kraken_cmc',
                                          'kucoin': 'exchange_pairs_kucoin_cmc'})
ex_final_df = ex_vol_df.merge(ex_pairs_df,
                              on='date', 
                              how='outer',
                              validate='one_to_one')

# Add CMC to macro column names
col_names = list(macro_df.columns.values)
col_names.remove('date')
for col in col_names:
    macro_df = macro_df.rename(columns={col:col+'_cmc'})
    
# Merge together
final_macro_df = macro_df.merge(ex_final_df,
                                on='date',
                                how='outer',
                                validate='one_to_one')

# Confirm no rows are missing all observations
assert(final_macro_df.shape[0] == final_macro_df.dropna(how='all').shape[0])

# Confirm it is unique on date
assert(len(np.unique(final_macro_df.date.values)) == final_macro_df.shape[0])

# Convert date column
final_macro_df['date'] = pd.to_datetime(final_macro_df.date, format='%Y-%m-%d', utc=False)

# Drop earlier than 2015 or later than 2021
final_macro_df = final_macro_df[final_macro_df.date.dt.year >= 2015]
final_macro_df = final_macro_df[final_macro_df.date.dt.year <= 2021]

# Sort and reset index
final_macro_df = final_macro_df.sort_values(by='date', ignore_index=True)


In [142]:
# SAVE PANEL AND CW DATA TO DERIVED
panel_df.to_pickle('../3-data/derived/cmc_panel.pkl')
cw_df.to_pickle('../3-data/derived/cmc_cw.pkl')
final_macro_df.to_pickle('../3-data/derived/cmc_macro.pkl')

## (2) Clean CMC BTC and ETH data

In [60]:
import pandas as pd
import numpy as np

In [61]:
# READ IN DATA

df            = pd.read_csv('../3-data/raw/cmc_price_vol_mcap_btceth.csv')
cmc_covars_df = pd.read_csv('../3-data/raw/cmc_token_covars_btceth.csv')
macro_df      = pd.read_csv('../3-data/raw/cmc_macro_timeseries_btceth.csv')

In [62]:
# CLEAN PANEL 

# Make panel data unique on the ID columns
df = df[df.index.isin(df[['date', 'cmc_id']].drop_duplicates().index)]
cmc_covars_df = cmc_covars_df[cmc_covars_df.index.isin(cmc_covars_df[['date', 'cmc_id']].drop_duplicates().index)]

# Merge panel data together
cmc_df = df.merge(cmc_covars_df,
                  on=['date', 'cmc_id'],
                  how='outer',
                  validate='one_to_one')

# Rename columns to standard convention (with data source name in it)
cmc_df   = cmc_df.rename(columns = {'usd_per_token': 'usd_per_token_cmc',
                                    'usd_mcap': 'usd_mcap_cmc',
                                    'usd_volume_24h': 'usd_volume_24h_cmc',
                                    'num_market_pairs': 'num_market_pairs_cmc', 
                                    'circulating_supply': 'circulating_supply_cmc'})

# Ensure data has all dates and token IDs
first_date = np.min(cmc_df.date)
last_date  = np.max(cmc_df.date)
dates = list(pd.date_range(first_date, last_date, freq='D').strftime('%Y-%m-%d'))
ids   = np.repeat(np.array([1, 1027]), len(cmc_df)/2)
full_id_df = pd.DataFrame(data={'date': np.concatenate((dates, dates)), 'cmc_id': ids})
cmc_df = cmc_df.merge(full_id_df,
                      on=['date', 'cmc_id'],
                      how='outer', 
                      validate='one_to_one')

# Sort values and reset index
cmc_df = cmc_df.sort_values(by=['date', 'cmc_id'], 
                            ignore_index=True)

# Ensure correct date type
cmc_df['date'] = pd.to_datetime(cmc_df['date'], format='%Y-%m-%d', utc=False)

# Clean missing values
btc_df = cmc_df[cmc_df.cmc_id==1]
btc_df.loc[btc_df.date==np.min(btc_df.date), 'num_market_pairs_cmc'] = 1000
eth_df = cmc_df[cmc_df.cmc_id==1027]
eth_df.loc[eth_df.date==np.min(eth_df.date), 'num_market_pairs_cmc'] = 1000
cols = list(cmc_df.columns.values)
cols.remove('date')
cols.remove('cmc_id')
for col in cols:
    btc_df[col] = btc_df[col].interpolate(limit_direction='forward', axis=0, method='cubic')
    eth_df[col] = eth_df[col].interpolate(limit_direction='forward', axis=0, method='cubic')
cmc_df = pd.concat((btc_df, eth_df))
cmc_df = cmc_df.sort_values(by=['date', 'cmc_id'])

# Confirm no missings in the df
assert(cmc_df.isnull().sum().values.sum() == 0)

# Set column order
cmc_df = cmc_df[['date', 'cmc_id', 
                 'usd_per_token_cmc', 'usd_mcap_cmc', 'usd_volume_24h_cmc',
                 'num_market_pairs_cmc', 'circulating_supply_cmc']]

# Sort values and reset index
cmc_df = cmc_df.sort_values(by=['date', 'cmc_id'], 
                            ignore_index=True)

/tmp/ipykernel_153159/2930331424.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_df[col] = btc_df[col].interpolate(limit_direction='forward', axis=0, method='cubic')
/tmp/ipykernel_153159/2930331424.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eth_df[col] = eth_df[col].interpolate(limit_direction='forward', axis=0, method='cubic')


In [63]:
# CLEAN MACRO VARS

# Clean date column
macro_df['date'] = pd.to_datetime(macro_df.date, format='%Y-%m-%d', utc=False)

# Add CMC to macro column names
col_names = list(macro_df.columns.values)
col_names.remove('date')
for col in col_names:
    macro_df = macro_df.rename(columns={col:col+'_cmc'})
    
# Ensure it has all dates
first_date = np.min(macro_df.date)
last_date  = np.max(macro_df.date)
dates = list(pd.date_range(first_date, last_date, freq='D'))
dates_df = pd.DataFrame(data={'date': dates})
macro_df   = macro_df.merge(dates_df,
                            on=['date'],
                            how='outer', 
                            validate='one_to_one')
macro_df = macro_df.sort_values('date').reset_index(drop=True)
cols = list(macro_df.columns.values)
cols.remove('date')
for col in cols:
    macro_df[col] = macro_df[col].interpolate(limit_direction='forward', axis=0, method='cubic')
    
# Confirm no missings in the df
assert(macro_df.isnull().sum().values.sum() == 0)

# Confirm it is unique on date
assert(len(np.unique(macro_df.date.values)) == macro_df.shape[0])

# Sort and reset index
macro_df = macro_df.sort_values(by='date', ignore_index=True)


In [64]:
# SAVE PANEL AND CW DATA TO DERIVED
cmc_df.to_pickle('../3-data/derived/cmc_btceth_panel.pkl')
macro_df.to_pickle('../3-data/derived/cmc_btceth_macro.pkl')